In [ ]:
!pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 20.1 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm
from tqdm.auto import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Masking for EPT

Here we are using Ept data calculated using the `EPT calculation.ipynb`

In [ ]:
ds=xr.open_mfdataset('/content/drive/MyDrive/AP_HW/ERA5 Data/EPT/*.nc')
lat_range=slice(26.5,20.75)
lon_range=slice(88,92.5)
ds_ept=ds.ept.sel(latitude=lat_range,longitude=lon_range)
ds_ept_bdt=ds_ept.shift(time=6)
ds_ept_bdt

<xarray.DataArray 'ept' (time: 473376, latitude: 24, longitude: 19)> Size: 2GB
dask.array<rechunk-merge, shape=(473376, 24, 19), dtype=float64, chunksize=(8784, 24, 19), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 4MB 1971-01-01 ... 2024-12-31T23:00:00
  * latitude   (latitude) float32 96B 26.5 26.25 26.0 25.75 ... 21.25 21.0 20.75
  * longitude  (longitude) float32 76B 88.0 88.25 88.5 88.75 ... 92.0 92.25 92.5
Attributes:
    units:      K
    long_name:  Equivalent potential temperature

In [ ]:
mask_df=pd.read_csv('/content/drive/MyDrive/AP_HW/ERA5 Data/BD_region_mask.csv')
mask_df=mask_df[mask_df['mask']==1]
mask_df


,latitude,longitude,mask
1,26.50,88.25,1
2,26.50,88.50,1
4,26.50,89.00,1
21,26.25,88.25,1
22,26.25,88.50,1
...,...,...,...
436,21.25,92.00,1
437,21.25,92.25,1
438,21.25,92.50,1
457,21.00,92.25,1


In [ ]:
# Define the specific coordinates to mask out
coords_to_mask = list(mask_df[['latitude', 'longitude']].itertuples(index=False, name=None))

# Initialize the mask with False values
mask = np.zeros(ds_ept_bdt.shape, dtype=bool)


# Loop through each coordinate pair to create a mask
for lat, lon in tqdm(coords_to_mask):
    # Broadcast latitude and longitude masks
    lat_mask = ds_ept_bdt.latitude == lat
    lon_mask = ds_ept_bdt.longitude == lon

    # Create a 3D mask by combining latitude and longitude masks
    combined_mask = lat_mask & lon_mask
    combined_mask=np.stack([combined_mask]*ds_ept_bdt.sizes['time'])

    # Update the overall mask
    mask |= combined_mask

# Apply the mask to the dataset (set the masked region to NaN)
ds_ept_bdt_masked = ds_ept_bdt.where(mask)

ds_ept_bdt_masked

In [ ]:
ds_ept_bdt_masked.to_netcdf(f'/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/ept_bdt_masked.nc')


# Masking for t2m

In [ ]:
def get_masked(ds_t2m_bdt):
  mask_df=pd.read_csv('/content/drive/MyDrive/AP_HW/ERA5 Data/BD_region_mask.csv')
  mask_df=mask_df[mask_df['mask']==1]

  # Define the specific coordinates to mask out
  coords_to_mask = list(mask_df[['latitude', 'longitude']].itertuples(index=False, name=None))

  # Initialize the mask with False values
  mask = np.zeros(ds_t2m_bdt.shape, dtype=bool)


  # Loop through each coordinate pair to create a mask
  for lat, lon in tqdm(coords_to_mask):
      # Broadcast latitude and longitude masks
      lat_mask = ds_t2m_bdt.latitude == lat
      lon_mask = ds_t2m_bdt.longitude == lon

      # Create a 3D mask by combining latitude and longitude masks
      combined_mask = lat_mask & lon_mask
      combined_mask=np.stack([combined_mask]*ds_t2m_bdt.sizes['time'])

      # Update the overall mask
      mask |= combined_mask

  # Apply the mask to the dataset (set the masked region to NaN)
  ds_t2m_bdt_masked = ds_t2m_bdt.where(mask)
  return ds_t2m_bdt_masked



Here we are using ERA5 data to calculate our **daily maximum t2m**.

In [ ]:
import os
from tqdm.auto import tqdm
directory_path='/content/drive/MyDrive/AP_HW/ERA5 Data/t2m_d2m_sp'

t2m_list=[]
for filename in tqdm(sorted(os.listdir(directory_path))):
  if filename.endswith('.nc'):
    filepath=os.path.join(directory_path,filename)
    ds=xr.open_dataset(filepath)
    year=int(filename[11:-3])
    lat_range=slice(26.5,20.75)
    lon_range=slice(88,92.5)
    ds_t2m=ds.t2m.sel(latitude=lat_range,longitude=lon_range)
    ds_t2m_bdt=ds_t2m.shift(time=6)
    ds_t2m_bdt_masked=get_masked(ds_t2m_bdt)
    ds_t2m_bdt_masked=ds_t2m_bdt_masked.to_dataset()
    t2m_list.append(ds_t2m_bdt_masked)
masked_t2m_all=xr.concat(t2m_list,dim='time')
masked_t2m_all.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/t2m_bdt_masked.nc')

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/257 [00:00<?, ?it/s]

...

  0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/257 [00:00<?, ?it/s]